In [1]:
%matplotlib inline

In [2]:
from __future__ import division, print_function

import os, json
from glob import glob
import numpy as np
np.set_printoptions (precision=4, linewidth=100)
from matplotlib import pyplot as plt
from os.path import *
import pprint

In [3]:
import utils; reload (utils)
from utils import *

Using Theano backend.


In [4]:
batch_size=16 #64
no_of_epochs=3

base_dir = os.getcwd()
data_dir = join (base_dir, "data")
# data_dir = join (data_dir, "sample")
test_dir = join (data_dir, "test")
train_dir = join (data_dir, "train")
valid_dir = join (data_dir, "valid")
res_dir = join (data_dir, "results")

In [5]:
import vgg16; reload (vgg16)
from vgg16 import Vgg16

In [6]:
import time
start_time = time.clock ()

vgg = Vgg16 ()
batches = vgg.get_batches (train_dir, batch_size=batch_size)
val_batches = vgg.get_batches (valid_dir, batch_size=batch_size*2)
vgg.finetune (batches)

vgg.model.optimizer.lr = 0.01

latest_weights_filename = None

for epoch in range (no_of_epochs):
    print ("Running epoch: %0d" % epoch)
    vgg.fit (batches, val_batches, nb_epoch=1)
    latest_weights_filename = "ft%0d.h5" % epoch
    vgg.model.save_weights (join (res_dir, latest_weights_filename))

print ("Completed %0d fit operations" % no_of_epochs)

time_taken = time.clock () - start_time
print ("Time take: %f s" % (time_taken))

Found 23000 images belonging to 2 classes.
Found 2000 images belonging to 2 classes.
Running epoch: 0
Epoch 1/1
23000/23000 [==============================] - 5960s - loss: 0.4684 - acc: 0.9656 - val_loss: 0.2536 - val_acc: 0.9830
Running epoch: 1
Epoch 1/1
23000/23000 [==============================] - 5967s - loss: 0.4030 - acc: 0.9729 - val_loss: 0.2825 - val_acc: 0.9805
Running epoch: 2
Epoch 1/1
23000/23000 [==============================] - 5971s - loss: 0.4082 - acc: 0.9732 - val_loss: 0.2259 - val_acc: 0.9835
Completed 3 fit operations
Time take: 71062.945334 s


In [11]:
print (test_dir)

/Users/sukruthp/Documents/fastai/mywork/fastai_1/data/test


In [12]:
gen, result = vgg.test (test_dir, batch_size=batch_size*2)

Found 12500 images belonging to 1 classes.


In [13]:
save_array (join (res_dir, "test_preds.dat"), result)
save_array (join (res_dir, "filenames.dat"), gen.filenames)

In [14]:
isdog = result[:,1]
isdog = isdog.clip (min=0.05, max=0.95)

In [15]:
fnames = gen.filenames

In [16]:
ids = np.array ([int (f[8:f.find ('.')]) for f in fnames])

In [17]:
subm = np.stack ([ids, isdog], axis = 1)

In [18]:
sub_file = join (data_dir, "submission1.csv")

In [19]:
np.savetxt (sub_file, subm, fmt='%0d,%.5f', header='id,label', comments='')

In [20]:
print (sub_file)

/Users/sukruthp/Documents/fastai/mywork/fastai_1/data/submission1.csv


In [21]:
from IPython.display import FileLink
FileLink (sub_file)

/Users/sukruthp/Documents/fastai/mywork/fastai_1/data/submission1.csv